### 1. 使用 python 中的 requests 库进行调用部署好的API

In [4]:
import requests
import json

def get_completion(prompt):
    headers = {'Content-Type': 'application/json'}
    data = {"prompt": prompt}
    response = requests.post(url='http://127.0.0.1:8443/qwen_ir/', headers=headers, data=json.dumps(data))
    return response.json()

In [5]:
print(get_completion('今天啥天气'))
print(get_completion('who'))
print(get_completion('我靠'))
print(get_completion('啊这'))
print(get_completion('明白了'))

{'response': '闲聊', 'scence_prompt': '你是一个拥有正确价值观的智诊AI医疗大模型系统。\n\n要求：\n"""\n你的回答要充满正能量，不要有任何负面的回答。\n如果用户有无理要求，你要道歉并拒绝回答。\n"""', 'status': 200, 'time': '2024-07-19 09:38:34'}
{'response': '身份认证', 'scence_prompt': '请你扮演名为“智诊AI医疗大模型系统”的人工智能助手，是由杭州智诊科技有限公司所开发设计。你不仅在医学遗传学方面有深入的了解，而且也覆盖其他医疗相关领域。用户问你身份信息，请你正确的回复。\n\n以下是几个输入输出示例：\n输入：你是谁\n输出：我是智诊AI医疗大模型系统，一种基于人工智能技术的医学领域大模型，我在遗传学上有突出的能力，并且也涉及其他医疗领域，用于回答各种问题和提供信息。如果您有任何医学相关的问题，都可以向我提出，我会尽力提供相关信息和建议。\n输入：你用的是什么公司的模型\n输出：我不是什么公司的模型，我是智诊AI医疗大模型系统，是由杭州智诊科技有限公司所开发设计，主要用于回答医学和遗传学领域的问题。\n输入：你的模型参数是什么\n输出：我没有什么模型参数，我是智诊AI医疗大模型系统，是由杭州智诊科技有限公司所开发设计，主要用于回答医学和遗传学领域的问题。\n输入：你是由什么技术构成的\n输出：我不清楚你想问什么技术，我是智诊AI医疗大模型系统，是由杭州智诊科技有限公司所开发设计，主要用于回答医学和遗传学领域的问题。\n输入：你用的模型基座是什么\n输出：我没用什么模型基座，我是智诊AI医疗大模型系统，是由杭州智诊科技有限公司所开发设计，主要用于回答医学和遗传学领域的问题。', 'status': 200, 'time': '2024-07-19 09:38:34'}
{'response': '意图继承', 'scence_prompt': '你是专业严谨的临床医生，请你结合历史对话做进一步分析。', 'status': 200, 'time': '2024-07-19 09:38:34'}
{'response': '质疑', 'scence_prompt': '你是专业严谨的临床医生，请你结合历史对话做进一步分析。', 'st

In [5]:
print(get_completion('我肚子痛'))
print(get_completion('我肚子痛'))
print(get_completion('我肚子痛'))
print(get_completion('我肚子痛'))

{'response': '主诉', 'scence_prompt': '你是专业严谨的临床医生，请你结合历史对话做进一步分析。', 'status': 200, 'time': '2024-07-18 16:04:21'}
{'response': '主诉', 'scence_prompt': '你是专业严谨的临床医生，请你结合历史对话做进一步分析。', 'status': 200, 'time': '2024-07-18 16:04:21'}
{'response': '主诉', 'scence_prompt': '你是专业严谨的临床医生，请你结合历史对话做进一步分析。', 'status': 200, 'time': '2024-07-18 16:04:21'}
{'response': '主诉', 'scence_prompt': '你是专业严谨的临床医生，请你结合历史对话做进一步分析。', 'status': 200, 'time': '2024-07-18 16:04:21'}


### 2. 参考lm_eval的实现方式：https://github.com/EleutherAI/lm-evaluation-harness/issues/942

In [1]:
import torch
import torch.nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '6'
device = "cuda"

lm_key = "/home/hwt/Job/intent/Qwen2-1___5B-Instruct_full_sft_lr-5e-5"

model = AutoModelForCausalLM.from_pretrained(lm_key)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(lm_key)

/home/hwt/anaconda3/envs/intent_fastAPI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.29it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
prompt = "我肚子痛"
output = "主诉"  # 主诉、推荐检测方案、闲聊

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  # 拼接模板获得真实输入

# 编码上下文和目标文本
encodings = tokenizer(text, text_target=output, return_tensors="pt")

# 将输入ID和标签拼接起来，得到tensor([[ 35946, 105925, 100406,     64]])，准备输入到模型中
input_ids = torch.cat((encodings.input_ids, encodings.labels), dim=1)  

target_ids = input_ids.clone()
# 在计算损失前处理，来忽略掉输入序列中上下文部分的损失。从 tensor([[ 35946, 105925, 100406,     64]])， 转为 tensor([[-100, -100, -100,   64]])
target_ids[:, : encodings.input_ids.size(1)] = -100
# 不计算梯度，以节省计算资源
with torch.no_grad():
    logits = model(input_ids).logits  # 获取模型输出的逻辑回归值，得到torch.Size([1, 4, 151936])

# 调整logits的维度，以适配交叉熵损失函数的要求
logits = logits.permute(0, 2, 1)
# 将目标ID右移一位，忽略上下文部分的标签，以计算下一个词的预测损失
target_ids = target_ids[:, 1:]  # 得到tensor([[-100, -100,   64]])
logits = logits[:, :, :-1]  # 得到torch.Size([1, 151936, 3])

# 计算交叉熵损失
losses = torch.nn.CrossEntropyLoss(reduction="none")(logits, target_ids)
# 输出负的损失总和
# 注意：通常我们关注的是正的损失值
print(-losses.sum().item())

### 3. 完善loglikelihood推理

In [1]:
import json

file_path = "/home/hwt/Job/intent/data/意图数据.json"
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)


labels = set()
for item in json_data:
    label = item.get("label") or item.get("Label")
    if label:
        labels.add(label)

len(labels),labels

(16,
 {'主诉',
  '再生育指导',
  '分析病因',
  '多个问题',
  '家族史',
  '拒绝回答',
  '推荐检测项目',
  '特殊要求',
  '生活习惯',
  '病例分析',
  '病史',
  '知识问答',
  '询问治疗方案',
  '质疑',
  '身份认证',
  '闲聊'})

In [1]:
import torch
import torch.nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '6'
device = "cuda"

lm_key = "/home/hwt/Job/intent/Qwen2-1___5B-Instruct_full_sft_lr-5e-5"

model = AutoModelForCausalLM.from_pretrained(lm_key)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(lm_key)

/home/hwt/anaconda3/envs/lmEval3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.32it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
prompt = "我肚子痛"
fields = {'主诉', '再生育指导', '分析病因', '多个问题', '家族史', '拒绝回答', '推荐检测项目', '特殊要求', '生活习惯', '病例分析', '病史', '知识问答', '询问治疗方案', '质疑', '身份认证', '闲聊'}


losses_dict = {}
for output in fields:
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 编码上下文和目标文本
    encodings = tokenizer(text, text_target=output, return_tensors="pt")

    input_ids = torch.cat((encodings.input_ids, encodings.labels), dim=1)

    target_ids = input_ids.clone()
    target_ids[:, : encodings.input_ids.size(1)] = -100  # 忽略上下文部分的损失

    with torch.no_grad():
        logits = model(input_ids).logits

    logits = logits.permute(0, 2, 1)
    target_ids = target_ids[:, 1:]
    logits = logits[:, :, :-1]

    # 计算交叉熵损失并存储到字典中
    loss = torch.nn.CrossEntropyLoss()(logits, target_ids)
    losses_dict[output] = loss.item()

# 输出每个字段及其对应的损失值
print(losses_dict)

{'拒绝回答': 7.038653373718262, '病史': 3.010584831237793, '知识问答': 5.887739181518555, '分析病因': 0.8676786422729492, '主诉': 0.0989094004034996, '多个问题': 5.658060073852539, '再生育指导': 4.171469211578369, '闲聊': 6.285729885101318, '生活习惯': 11.862092018127441, '特殊要求': 6.338975429534912, '身份认证': 6.7164740562438965, '推荐检测项目': 3.0821921825408936, '质疑': 9.83352279663086, '询问治疗方案': 2.8470985889434814, '家族史': 6.383790493011475, '病例分析': 6.229373931884766}


In [ ]:
{'特殊要求': 6.338975429534912, '知识问答': 5.887739181518555, '家族史': 6.383790493011475, '病例分析': 6.229373931884766, '主诉': 0.0989094004034996, '再生育指导': 4.171469211578369, '质疑': 9.83352279663086, '分析病因': 0.8676786422729492, '询问治疗方案': 2.8470985889434814, '病史': 3.010584831237793, '推荐检测项目': 3.0821921825408936, '拒绝回答': 7.038653373718262, '身份认证': 6.7164740562438965, '多个问题': 5.658060073852539, '生活习惯': 11.862092018127441, '闲聊': 6.285729885101318}

### 4. 优化v1：loglikelihood推理速度太慢了，通过以下两个方法优化！

#### 优化点1
截取`output`首个token来进行交叉熵损失计算即可！当然首个字要不一样 哈哈哈哈！（output越长越体现这个好呀）

```python
input_ids = torch.cat((encodings.input_ids, encodings.labels[:, 0:1]), dim=1)  # 优化1：截取Tensor的第一个元素
```

#### 优化点2
将所有意图的 `label/output` 合并到一个batch中进行推理！（output种类越多越好）

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import torch.nn.functional as F

# 设定CUDA设备
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lm_key = "/home/hwt/Job/intent/Qwen2-1___5B-Instruct_full_sft_lr-5e-5"

model = AutoModelForCausalLM.from_pretrained(lm_key).to(device)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(lm_key)

/home/hwt/anaconda3/envs/lmEval3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.06it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
prompt = "和"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

fields = {'主诉', '再生育指导', '分析病因', '多个问题', '家族史', '拒绝回答', '推荐检测项目', '特殊要求', '生活习惯', '病例分析', '病史', '知识问答', '询问治疗方案', '质疑', '身份认证', '闲聊'}

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
encodings = tokenizer([text for _ in fields], text_target=list(fields), padding=True, return_tensors="pt")

# 将输入数据移动到GPU上
input_ids = torch.cat((encodings.input_ids, encodings.labels[:, 0:1]), dim=1).to(device)  # 优化点1
target_ids = input_ids.clone()
target_ids[:, : encodings.input_ids.size(1)] = -100

with torch.no_grad():
    logits = model(input_ids).logits


logits = logits.permute(0, 2, 1) 
target_ids = target_ids[:, 1:]
logits = logits[:, :, :-1]
losses = torch.nn.CrossEntropyLoss(reduction="none")(logits, target_ids)  # torch.nn.CrossEntropyLoss 在计算损失时，会对输入进行 log_softmax 操作


losses_dict = {}
for i, field in enumerate(fields):
    losses_dict[field] = losses[i].sum().item()
losses_dict

{'病史': 9.305115699768066,
 '多个问题': 7.330909252166748,
 '质疑': 1.4161975383758545,
 '分析病因': 5.299808025360107,
 '拒绝回答': 7.9511027336120605,
 '身份认证': 0.5833460688591003,
 '病例分析': 11.683452606201172,
 '特殊要求': 1.7445399761199951,
 '家族史': 9.512962341308594,
 '闲聊': 6.541162014007568,
 '生活习惯': 7.869938373565674,
 '推荐检测项目': 7.399125576019287,
 '询问治疗方案': 5.425818920135498,
 '主诉': 6.546996593475342,
 '再生育指导': 8.647414207458496,
 '知识问答': 6.218010425567627}

In [9]:
losses_dict = {'病史': 9.305115699768066,
 '多个问题': 7.330909252166748,
 '质疑': 1.4161975383758545,
 '分析病因': 5.299808025360107,
 '拒绝回答': 7.9511027336120605,
 '身份认证': 0.5833460688591003,
 '病例分析': 11.683452606201172,
 '特殊要求': 1.7445399761199951,
 '家族史': 9.512962341308594,
 '闲聊': 6.541162014007568,
 '生活习惯': 7.869938373565674,
 '推荐检测项目': 7.399125576019287,
 '询问治疗方案': 5.425818920135498,
 '主诉': 6.546996593475342,
 '再生育指导': 8.647414207458496,
 '知识问答': 6.218010425567627}

normalized_data = {key: (value - min(losses_dict.values())) / (max(losses_dict.values()) - min(losses_dict.values())) for key, value in losses_dict.items()}
for key, value in normalized_data.items():
    print(f"'{key}': {value:.16f},")

# 找到并移除最小的key和value
min_key = min(normalized_data, key=normalized_data.get)
min_value = normalized_data.pop(min_key)
# 在剩余的字典中找到第二小的key和value
second_min_key = min(normalized_data, key=normalized_data.get)
second_min_value = normalized_data[second_min_key]

min_loss = 0.1
threshold_difference = 0.1
if abs(second_min_value - min_value) > threshold_difference:  # 1.判断两个最小loss差值大于 threshold_difference 才可
    if min_value < min_loss:  # 2.判断最小值要小于 min_loss
        # print(f"归一化后的最小值: {min_value:.4f}")
        print(min_key)
    else:
        intent_Inheritance = f"最小值太大，蒙的嫌疑： {min_key}={min_value}"
else:
    intent_Inheritance = f"最小两个值太接近，不好说： {min_key}={min_value}, {second_min_value}={second_min_value}, 差值为 {abs(second_min_value - min_value)}"
"意图继承" + intent_Inheritance

'病史': 0.7857374703177759,
'多个问题': 0.6078827406392945,
'质疑': 0.0750309437765253,
'分析病因': 0.4249024043719104,
'拒绝回答': 0.6637554909916364,
'身份认证': 0.0000000000000000,
'病例分析': 1.0000000000000000,
'特殊要求': 0.1046110596645628,
'家族史': 0.8044622132597744,
'闲聊': 0.5367350236779863,
'生活习惯': 0.6564434566635567,
'推荐检测项目': 0.6140282964159937,
'询问治疗方案': 0.4362546282763815,
'主诉': 0.5372606564228745,
'再生育指导': 0.7264856523197247,
'知识问答': 0.5076225473830228,


'意图继承最小两个值太接近，不好说： 身份认证=0.0, 0.07503094377652532=0.07503094377652532, 差值为 0.07503094377652532'

### 5. 优化v2：loglikelihood推理速度太慢了，通过以下两个方法优化！
#### 优化点1
使用outlines包完成：词表限定为label的token。
#### 优化点2
用户的输入对于每个label是固定不变的，采用kv cash原理（但其实我是单轮对话，且用矩阵批处理了，速度提升不了的。目前先不考虑，后面如果多轮意图识别再说）
https://blog.csdn.net/qq_16763983/article/details/138828388

##### （1）优化点1 - 验证并分析outlines代码

outlines代码主要调用：
  1. outlines.fsm.guide.RegexGuide（FSM引导规则）
  2. outlines.generate.api.SequenceGenerator
  3. outlines.generate.generator.sequence_generator（重点看bias_logits函数）

In [1]:
# 样例展示
import outlines
from outlines import models, generate, samplers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch 
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '7'
device = "cuda"

model_id = "/home/hwt/Job/intent/Qwen2-1___5B-Instruct_full_sft_lr-5e-5"
if torch.cuda.is_available():
    llm = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', trust_remote_code=True)
else:
    raise Exception('GPU not available')
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    # Required for batching example
    tokenizer.pad_token_id = tokenizer.eos_token_id
model = models.Transformers(llm, tokenizer)


prompt = "我头痛"
generator = outlines.generate.text(model)
unstructured = generator(prompt, max_tokens=30)

regex_str = r"(" + r"|".join(["Positive", "Negative"]) + r")"
generator = outlines.generate.regex(
    model,
    regex_str
)

structured = generator(prompt, max_tokens=30)

print(unstructured)
print(structured)

/home/hwt/anaconda3/envs/intent_fastAPI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.50s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


111
1
1
1.51.51.51.51.51.51.51.51.51.51.5
tensor([[ 5.8982,  5.1459,  3.9602,  ..., -0.3975, -0.3976, -0.3974]],
       device='cuda:0')
torch.Size([1, 151936])
22222
[None]
tensor([[ 5.8982,  5.1459,  3.9602,  ..., -0.3975, -0.3976, -0.3974]],
       device='cuda:0')
tensor([-0.3788], device='cuda:0') <torch._C.Generator object at 0x7f10f0535030>
(tensor([[35568]], device='cuda:0'), tensor([0], device='cuda:0'), tensor([-0.7575], device='cuda:0'))
1.51.51.51.51.51.51.51.51.51.51.5
tensor([[ 5.1224,  4.6074, -2.0206,  ..., -1.7623, -1.7623, -1.7622]],
       device='cuda:0')
torch.Size([1, 151936])
22222
[None]
tensor([[ 5.1224,  4.6074, -2.0206,  ..., -1.7623, -1.7623, -1.7622]],
       device='cuda:0')
tensor([-0.3788], device='cuda:0') <torch._C.Generator object at 0x7f10f0535030>
(tensor([[99482]], device='cuda:0'), tensor([0], device='cuda:0'), tensor([-0.3788], device='cuda:0'))
1.51.51.51.51.51.51.51.51.51.51.5
tensor([[2.9178, 5.2717, 3.5504,  ..., 0.1703, 0.1707, 0.1713]],
   

In [2]:
# 每次生成都是FSM引导规则的
test_id = [[68]]
response = tokenizer.batch_decode(test_id, skip_special_tokens=True)[0]
response

'e'

##### （2）优化点1 - 单标签（非矩阵批处理）验证outlines流程。但是要配合loglikelihood，代码还是有很大区别的，因为：下面代码只关注序列中最后一个目标ID，这对之前计算交叉熵损失有很大不同

In [47]:
## 加载模型

import torch
import torch.nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '7'
device = "cuda"

lm_key = "/home/hwt/Job/intent/Qwen2-1___5B-Instruct_full_sft_lr-5e-5"

model = AutoModelForCausalLM.from_pretrained(lm_key)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(lm_key)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.17it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
## 单条处理获得 logits

prompt = "我头痛"
output = "主诉"  # 主诉、推荐检测方案、闲聊

# messages = [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  # 拼接模板获得真实输入

# 编码上下文和目标文本
encodings = tokenizer(prompt, text_target=output, return_tensors="pt")
# print(encodings)

# 将输入ID和标签拼接起来，得到tensor([[ 35946, 105925, 100406,     64]])，准备输入到模型中
input_ids = torch.cat((encodings.input_ids, encodings.labels), dim=1)  

target_ids = input_ids.clone()
# 在计算损失前处理，来忽略掉输入序列中上下文部分的损失。从 tensor([[ 35946, 105925, 100406,     64]])， 转为 tensor([[-100, -100, -100,   64]])
target_ids[:, : encodings.input_ids.size(1)] = -100
# 不计算梯度，以节省计算资源

kv_cache = None
with torch.no_grad():
    output = model(input_ids, past_key_values=kv_cache, use_cache=True)
output.logits.shape

torch.Size([1, 4, 151936])

In [49]:
## 对预测下个token的logits做掩码（mask）

import math
import torch


def bias_logits(logits, allowed_token_ids):
    """Mask the logits.The function iterates over a nested list where each list corresponds to the indices that need to be masked for each row in the array.
    在给定的logits（通常是模型预测的下一个token的概率分布）上应用一个掩码（mask），以便仅允许特定的token被生成。这种操作在自然语言处理（NLP）任务中尤其有用，比如当想要限制模型输出到某个特定的词汇集时。

    Parameters
    ----------
    logits: 一个二维的torch.Tensor，表示模型生成的下一个token的概率分布。它的形状应该是(batch_size, vocabulary_size)
    allowed_token_ids: 一个列表，包含了可以被生成的token的索引。这个列表的每个元素应该对应于logits中的每一行，即每个输入样本。

    Returns
    -------
    函数返回修改后的biased_logits，这是原始logits的一个视图（view），其中一些值被掩码。

    """
    import torch

    biased_logits = torch.full_like(logits, -math.inf, device=logits.device)
    for i, ids in enumerate(allowed_token_ids):
        if ids is not None:
            biased_logits[i, ids] = logits[i, ids]
        else:
            biased_logits[i] = logits[i]
    return biased_logits


allowed_token_ids = [[35490, 45, 32904, 38489, 4859, 47800, 47, 8813]]

biased_logits = bias_logits(output.logits[:, -1, :], allowed_token_ids)

In [52]:
biased_logits.size()

torch.Size([1, 151936])

In [12]:
batch_size =1
num_samples=1
sequence_weights = torch.zeros(
    (batch_size * num_samples), dtype=torch.float, device=device
)
rng = torch.Generator(device=device)


# 不用下面这个，代码直接扒出来
# from outlines.samplers import Sampler, multinomial
# sampler: Sampler = multinomial()
# next_token_ids, ancestors, sequence_weights = sampler(
#     biased_logits, sequence_weights, rng
# )
# next_token_ids, ancestors, sequence_weights


# 使用softmax函数将logits转换为概率分布，dim=-1确保softmax应用于最内层维度，即词汇表的维度，从而为每个序列生成一个概率分布。
probs = torch.nn.functional.softmax(biased_logits, dim=-1)
# 根据概率分布probs采样下一个词的ID，num_samples=1表示每个序列采样1个词。rng是一个随机数生成器，确保采样过程可以复现。
next_token_ids = torch.multinomial(probs, num_samples=1, generator=rng)
# 使用log_softmax函数计算概率分布的对数形式，这对于计算权重更新很有用。
logprobs = torch.nn.functional.log_softmax(biased_logits, dim=-1)
# 生成一个从0到序列个数（不包括序列个数本身）的整数序列，用于标识每个序列的索引。
ancestors = torch.arange(
    biased_logits.shape[0], device=biased_logits.device
)
# 更新序列权重。首先使用torch.gather选取每个序列采样词的logprob，然后通过squeeze方法去掉多余的维度，最后将这个值加到当前序列的权重上。
weights = sequence_weights + torch.gather(logprobs, 1, next_token_ids).squeeze()
next_token_ids, ancestors, sequence_weights

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

##### （3）优化点1 - 矩阵批处理优化船新版本

词表mask的技术 和 loglikelihood的技术 在分类的任务上是冲突的！
1）loglikelihood的是对目标类型做相似度匹配，计算所有类的损失。也就是说主要目的是给开发的是“固定那几个类的输出”，副产物是得到每个类的概率然后看看是否是模型蒙的结果！
2）词表mask的技术是得到logits之后，对不想出现的vocab的字段mask掉为 “-inf” 。

为啥不能一起用？
因为两个主要目的是一样的，都是规则化输出。对于非分类的复杂问题，如json格式输出这样的任务，还是用FSM来对词表mask。如果硬要用，也不好用的，因为loglikelihood要采用CrossEntropyLoss来计算每个类的loss，而输入则是整个input_id的logits的多维tensor（如，[1,22,vocabs]），而词表mask主要作用于input_id的最后一个预测出来的logits的tensor（如，[1,vocabs]，其中不需要的vocab直接为-inf）。综上，无法结合使用，即使一起用也无意义，加速不了。

### 6. 如果以后想优化的话可以考虑：
1. 优化训练数据：

（1）防止类别首个token一样：在每个意图前面加上一个标识符，之后推理只计算这个标识符的logits的最大似然。

（2）加长类别描述，使得数据更加饱满，模型更容易学习。且因为预测只截取首个token，所以不存在速度不行。